In [10]:
import ibmdata

In [15]:
DAYSBACK = 300
QUERY = f"""
SELECT lot_id, wafer_id, last_test_date AS date, radius_Center_5 AS ERZONE, family_code,
    corner || ' ' || categoryName as parameter, SUM(count) AS Count, count(*) AS Num_chips
FROM DMIW.PattRecChipFactR prcfr
JOIN DMIW_SYSTEMS.DerivedDataSetup dds ON dds.derivedSetupKey = prcfr.derivedSetupKey
JOIN DMIW_SYSTEMS.PattRecCategory prc ON prc.pattRecCatKey = prcfr.pattRecCatKey
JOIN DMIW_SYSTEMS.Geography g ON g.geographyKey = prcfr.geographyKey
JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = prcfr.testedWaferKey
WHERE Last_test_date >= (current date - {DAYSBACK} days) 
AND Tech_id = '5HPP' 
--AND tw.Calcdefs in ('PEL11', 'PEL21', 'PEL31')
--AND corner IN ('PEL1L3March90p75','PEL1K1March90p75')
--AND (categoryName = 'BIT' OR categoryName = 'DBC' OR categoryName = 'PERF')
AND exclude_Flag = 'N'
GROUP BY lot_id, wafer_id, last_test_date, radius_Center_5, family_code, corner || ' ' || categoryName
"""

In [16]:
ibmdata.isdw.query(QUERY)

2023-11-25 06:10:03,574 INFO     ibmdata   : retrieved (30651, 8) (rows, cols) of data in 0:00:22.031975 from ISDW


,lot_id,wafer_id,date,erzone,family_code,parameter,count,num_chips
0,AZDU1D.1,AZDU1-01,2023-02-20,A,5A,M026Marchs91p05A BIT,53,7
1,AZDU1D.1,AZDU1-01,2023-02-20,B,5A,M026Marchs91p05A BIT,330,20
2,AZDU1D.1,AZDU1-01,2023-02-20,C,5A,M026Marchs91p05A BIT,88,20
3,AZDU1D.1,AZDU1-01,2023-02-20,D,5A,M026Marchs91p05A BIT,21,9
4,AZDU1D.1,AZDU1-01,2023-02-20,E,5A,M026Marchs91p05A BIT,582,6
...,...,...,...,...,...,...,...,...
30646,FZ50T.1,FZ50T-25,2023-11-14,A,M1,M026VWBMarch91p05A PERF,64,8
30647,FZ50T.1,FZ50T-25,2023-11-14,B,M1,M026VWBMarch91p05A PERF,176,22
30648,FZ50T.1,FZ50T-25,2023-11-14,C,M1,M026VWBMarch91p05A PERF,304,38
30649,FZ50T.1,FZ50T-25,2023-11-14,D,M1,M026VWBMarch91p05A PERF,424,53


In [ ]:
-- sql for PATTREC retrieval 
SELECT
  lot_Id,
  last_fab_lot_id,
  wafer_Id,
  normalized_TestX AS chipX,
  normalized_TestY AS chipY,
  family_Code,
  lot_Grade,
  date,
  LEVEL,
  testedWafer.calcdefs,
  testedWafer.testProgramName AS step,
  last_Eqp_Id AS tool,
  corner || ' ' || categoryName AS PARAMETER,
  count AS parmValue,
  0.0 AS yield, 
   unitcell_X AS kerf_X,
  unitcell_Y AS kerf_Y,
  prodChip_LL_X +((prodChip_UR_X-prodChip_LL_X)/ 2) AS locX,
  prodChip_LL_Y +((prodChip_UR_Y-prodChip_LL_Y)/ 2) AS locY,
  area_Center_5,
  radius_Center_5,
  quadrant,
  locationType, 
   0 AS testparmkey, 
   geography.geographyKey AS dmiwKey
FROM
  DMIW.PattRecChipFactR pattRecChipFactR, 
   DMIW_SYSPRC.DerivedDataSetup derivedDataSetup, 
   DMIW_SYSPRC.PattRecCategory pattRecCategory, 
   DMIW_SYSPRC.Geography geography, 
   (
  SELECT
    testedWaferKey,
    lot_Id,
    last_fab_lot_id,
    wafer_Id,
    family_Code,
    cur_Lot_Grade AS lot_Grade,
    last_TestTimeStamp AS date,
    LEVEL,
    '' AS calcdefs,
    testProgramName,
    last_Eqp_Id
  FROM
    DMIW_SYSPRC.testedwafer testedwafer
  WHERE ( Last_test_date >= (current date - 15 days) and Tech_id = '5HPP' and Lot_id = 'FZ50T.1' )
  GROUP BY
    testedWaferKey,
    lot_Id,
    last_fab_lot_id,
    wafer_Id,
    family_Code,
    cur_Lot_Grade,
    last_TestTimeStamp,
    LEVEL,
    calcdefs,
    testProgramName,
    last_Eqp_Id) testedWafer
WHERE
  testedWafer.testedWaferKey = pattRecChipFactR.testedWaferKey
  AND 
   derivedDataSetup.derivedSetupKey = pattRecChipFactR.derivedSetupKey
  AND 
   pattRecCategory.pattRecCatKey = pattRecChipFactR.pattRecCatKey
  AND 
   geography.geographyKey = pattRecChipFactR.geographyKey
  AND 
   categoryName <> ''
  AND exclude_Flag = 'N'
  
order by lot_Id, wafer_Id, no

In [ ]:
SELECT wafer_Id as summaryLevel, lot_Id as lot_Id, wafer_Id, family_Code, lot_Grade, 
  corner || ' ' || categoryName AS PARAMETER,
  LEVEL,
  testedWafer.calcdefs,
  testedWafer.testProgramName AS step,
  last_Eqp_Id AS tool, 
   date, 
   n_Sample AS n, 
   1 AS lotCount,
  1 AS waferCount, 
   yield, 
   count_WMean AS mean, 
   count_Std_Dev AS stdDev, 
   count_PTile05 AS ptile05, 
   count_PTile50 AS ptile50, 
   count_PTile95 AS ptile95, 
   wafer_Id AS summaryLevel, 
   0 AS testparmkey, 
   testedWafer.testedWaferKey AS dmiwKey
FROM
  DMIW.PattRecWaferFact pattRecWaferFact, 
   DMIW_SYSPRC.DerivedDataSetup derivedDataSetup, 
   DMIW_SYSPRC.PattRecCategory pattRecCategory, 
   (
  SELECT
    testedWaferKey,
    lot_Id,
    wafer_Id,
    family_Code,
    cur_Lot_Grade AS lot_Grade,
    last_TestTimeStamp AS date,
    LEVEL,
    '' AS calcdefs,
    testProgramName,
    last_Eqp_Id
  FROM
    DMIW_SYSPRC.testedwafer testedwafer
  WHERE ( Last_test_date >= (current date - 15 days) and Tech_id = '5HPP' )
  GROUP BY
    testedWaferKey,
    lot_Id,
    wafer_Id,
    family_Code,
    cur_Lot_Grade,
    last_TestTimeStamp,
    LEVEL,
    calcdefs,
    testProgramName,
    last_Eqp_Id) testedWafer
WHERE
  testedWafer.testedWaferKey = pattRecWaferFact.testedWaferKey
  AND 
   derivedDataSetup.derivedSetupKey = pattRecWaferFact.derivedSetupKey
  AND 
   pattRecCategory.pattRecCatKey = pattRecWaferFact.pattRecCatKey
  AND 
   categoryName <> ''
  AND exclude_Flag = 'N'
  AND (n_Sample) > 0
  AND count_WMean IS NOT NULL
  
order by lot_Id, wafer_Id, corner, categoryName, level, testedWafer.calcdefs
WITH ur

;
